# OpenAI Function Calling with LangChain

**Learning Objectives**:
- Understand the concept of OpenAI Function Calling.
- Learn how to define functions and their JSON schemas.
- Explore how to integrate function calling with LangChain using structured output parsers.
- Demonstrate handling function calls and processing responses.


In [ ]:
%pip install openai langchain langchain-openai python-dotenv --quiet

In [ ]:
from openai import OpenAI
import json
from os import getenv
from langchain.output_parsers import PydanticOutputParser
from langchain_core.messages import HumanMessage, SystemMessage
from pydantic import BaseModel, Field
from typing import List
from pprint import pprint
import ast

from dotenv import load_dotenv
load_dotenv()

In [ ]:
from google.colab import userdata
import os
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API')

## Function Callse _Without_ LangChain

In [ ]:

# Define the schedule_meeting function
def schedule_meeting(date: str, time: str, attendees: List[str]):
    # Connect to calendar service (mock implementation)
    return {
        "event_id": "1234",
        "status": "Meeting scheduled successfully!",
        "date": date,
        "time": time,
        "attendees": attendees
    }

# Map function names to functions
OPENAI_FUNCTIONS = {
    "schedule_meeting": schedule_meeting
}

In [ ]:
# This is the OpenAI (not LangChain) way to define tools / functions
# Define the JSON schema for schedule_meeting
# see https://platform.openai.com/docs/guides/function-calling
functions = [
    {
        "type": "function",
        "function": {
            "type": "object",
            "name": "schedule_meeting",
            "description": "Set a meeting at a specified date and time for designated attendees",
            "parameters": {
                "type": "object",
                "properties": {
                    "date": {"type": "string", "format": "date"},
                    "time": {"type": "string", "format": "time"},
                    "attendees": {"type": "array", "items": {"type": "string"}},
                },
                "required": ["date", "time", "attendees"],
            },
        },
    }
]

**Explanation**:
- **JSON Schema**: Defines the structure of the function call, specifying the required parameters and their types. This schema helps the model understand how to format the function call.

In [ ]:
# We are still using the raw OpenAI API (not LangChain)

# Initialize the OpenAI client
client = OpenAI(api_key=getenv("OPENAI_API_KEY"))

# Start the conversation with a user request
messages = [
    {
        "role": "user",
        "content": '''Schedule a meeting on 2023-11-01 at 14:00 with Alice and Bob.''',
    }
]

# Send the conversation and function schema to the model
response = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=messages,
    tools=functions,
)

response = response.choices[0].message

In [ ]:
# The model should return the specs for a tool (function)
assert response.tool_calls

In [ ]:
response.tool_calls

In [ ]:

# Get the first function call
first_tool_call = response.tool_calls[0]

# Extract function name and arguments
function_name = first_tool_call.function.name
function_args = json.loads(first_tool_call.function.arguments)

print("This is the function name: ", function_name)
print("These are the function arguments: ", function_args)

In [ ]:
OPENAI_FUNCTIONS

In [ ]:
# Retrieve the function from the mapping
function = OPENAI_FUNCTIONS.get(function_name)

if not function:
    raise Exception(f"Function {function_name} not found.")

function

In [ ]:
# Call the function with the extracted arguments
function_response = function(**function_args)
function_response

In [ ]:
# Append the function's response to the messages
messages.append(
    {
        "role": "function", # <-- The other roles have been "Assistant" and "User"
        "name": function_name,
        "content": json.dumps(function_response),
    }
)

messages

In [ ]:
# Let the model generate a user-friendly response
second_response = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=messages
)

print(second_response.choices[0].message.content)

### Function Calling with Multiple Function Calls

Function calling can handle multiple function invocations within a single user request. Let's demonstrate this by scheduling two meetings in one conversation.

In [ ]:
# Start the conversation with a user request for two meetings
messages = [
    {
        "role": "user",
        "content": (
            '''Schedule a meeting on 2023-11-01 at 14:00 with Alice and Bob. '''
            '''Then I want to schedule another meeting on 2023-11-02 at 15:00 with Charlie and Dave.'''),
    }
]

# Send the conversation and function schema to the model
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
    tools=functions,
)

response = response.choices[0].message

for i, tool in enumerate(response.tool_calls, start=1):
    print(f"{i} : {tool.function}")

In [ ]:
# Check if the model wants to call functions
if response.tool_calls:
    for tool_call in response.tool_calls:
        # Extract function name and arguments
        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)

        # Retrieve the function from the mapping
        function = OPENAI_FUNCTIONS.get(function_name)

        if not function:
            raise Exception(f"Function {function_name} not found.")

        # Call the function with the extracted arguments
        # - Technically not in parallel but _could_ be
        function_response = function(**function_args)

        # Append the function's response to the messages
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": json.dumps(function_response),
            }
        )


    # Let the model generate a user-friendly response
    second_response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages
    )

    print(second_response.choices[0].message.content)

# Function Calling in LangChain

If you’d prefer to avoid writing JSON schemas and simply want to extract structured data from an LLM response, LangChain allows you to use function calling with Pydantic. This approach leverages `PydanticToolsParser` to parse the model's responses into defined Pydantic models, ensuring structured and reliable data extraction.
on.

The textbook is stale -- this has the right instructions for langchain  

https://python.langchain.com/v0.1/docs/modules/model_io/chat/function_calling/

In [ ]:
from langchain.output_parsers.openai_tools import PydanticToolsParser
from langchain_core.utils.function_calling import convert_to_openai_tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai.chat_models import ChatOpenAI
from pydantic import BaseModel, Field
from typing import Optional

In [ ]:
from langchain_core.tools import tool


@tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b



tools = [add, multiply]
tools

In [ ]:
llm = ChatOpenAI()

In [ ]:
llm_with_tools = llm.bind_tools(tools)

In [ ]:
query = "What is 3 * 12? Also, what is 11 + 49?"

llm_with_tools.invoke(query).tool_calls

There is little difference between a tool 'call' and esxtracting data.

We can use pydantic to create classes represnting the tools (functions to call)

In [ ]:
from pydantic import BaseModel, Field


# Note that the docstrings here are crucial, as they will be passed along
# to the model along with the class name.
class add(BaseModel):
    """Add two integers together."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


class multiply(BaseModel):
    """Multiply two integers together."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


tools = [add, multiply]

In [ ]:
from langchain_core.output_parsers.openai_tools import PydanticToolsParser

llm_with_tools = llm.bind_tools(tools)
chain = llm_with_tools | PydanticToolsParser(tools=tools)
chain.invoke(query)

Passing results back to the model (ToolMessage)

In [ ]:
from langchain_core.messages import HumanMessage, ToolMessage


@tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b


tools = [add, multiply]
llm_with_tools = llm.bind_tools(tools)

messages = [HumanMessage(query)]
ai_msg = llm_with_tools.invoke(messages)
messages.append(ai_msg)

TOOL_FUNCS = {
    'add': add,
    'multiply': multiply
}

for tool_call in ai_msg.tool_calls:
    selected_tool = tool_call["name"].lower()

    # Use 'invoke' because they are now tools, not functions.
    tool_output = TOOL_FUNCS[selected_tool].invoke(tool_call["args"])

    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

messages

In [ ]:
llm_with_tools.invoke(messages)

In [ ]:
# Define the Pydantic model for structured data extraction
class Article(BaseModel):
    """Identifying key points and contrarian views in an article."""

    points: str = Field(..., description="Key points from the article")
    contrarian_points: Optional[str] = Field(
        None, description="Any contrarian points acknowledged in the article"
    )
    author: Optional[str] = Field(None, description="Author of the article")



In [ ]:
_EXTRACTION_TEMPLATE = """Extract and save the relevant entities mentioned
in the following passage together with their properties.

If a property is not present and is not required in the function parameters,
do not include it in the output."""

# Create a prompt telling the LLM to extract information
prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage( _EXTRACTION_TEMPLATE),
        HumanMessage("{input}")
    ]
)

# Initialize the ChatOpenAI model
model = ChatOpenAI()

# Define Pydantic schemas
pydantic_schemas = [Article]

tools = [convert_to_openai_tool(schema) for schema in pydantic_schemas]

# Bind the tools directly to the LLM
model = model.bind_tools(tools=pydantic_schemas)

# Create an end-to-end chain with the parser
chain = prompt | model | PydanticToolsParser(tools=pydantic_schemas)

In [ ]:
# Example input text
input_text = """In the recent article titled 'AI adoption in industry,'
key points addressed include the growing interest in AI technologies across various sectors. However, the
author, Dr. Jane Smith, emphasizes the need for stringent regulations to prevent misuse."""

# Invoke the chain with the input text
result = chain.invoke(
    {
        "input": input_text
    }
)

# Print the structured extraction result
print(result)